In [43]:
import requests
import pandas as pd
import time

In [48]:
apikey = 'API_KEY'
headInfo = {
    'user-key': apikey
    }
#74, 170, 11052
city_ids = [11052]
len(city_ids)

1

In [49]:
#get cuisines
url = f'https://developers.zomato.com/api/v2.1/cuisines?city_id=11052'
data = requests.get(url, headers=headInfo)
cuisines = {cuisine['cuisine']['cuisine_id']:cuisine['cuisine']['cuisine_name'] for cuisine in data.json()['cuisines']}
len(cuisines)

79

In [50]:
#get establishment_types
url = f'https://developers.zomato.com/api/v2.1/establishments?city_id=11052'
data = requests.get(url, headers=headInfo)
establishments = {est['establishment']['id']:est['establishment']['name'] for est in data.json()['establishments']}
len(establishments)

18

In [51]:
#get categories
url = f'https://developers.zomato.com/api/v2.1/categories?'
data = requests.get(url, headers=headInfo)
categories = {cat['categories']['id']:cat['categories']['name'] for cat in data.json()['categories']}
len(categories)

13

In [52]:
def getJSON(city, cui, cat, est, start = 0, result= 'restaurants'):
    url = f'https://developers.zomato.com/api/v2.1/search?city_id={city}&cuisines={cui}&start={start}&count=20&category={cat}&establishment_type={est}'
    data = requests.get(url, headers=headInfo)
    return data.json()[result]

In [44]:
url = f'https://developers.zomato.com/api/v2.1/search?city_id=11052&start='
data = requests.get(url, headers=headInfo)
data.json()['restaurant']

{'results_found': 3824,
 'results_start': 0,
 'results_shown': 20,
 'restaurants': [{'restaurant': {'R': {'res_id': 18403125,
     'is_grocery_store': False,
     'has_menu_status': {'delivery': -1, 'takeaway': -1}},
    'apikey': '94ccd365dba11b2318f40110946a9aaf',
    'id': '18403125',
    'name': 'Gormeteria',
    'url': 'https://www.zomato.com/bandung/gormeteria-pasirkaliki?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1',
    'location': {'address': 'Jl. Pasirkaliki No. 176, Pasirkaliki, Bandung',
     'locality': 'Pasirkaliki',
     'city': 'Bandung',
     'city_id': 11052,
     'latitude': '-6.9015680000',
     'longitude': '107.5976120000',
     'zipcode': '',
     'country_id': 94,
     'locality_verbose': 'Pasirkaliki, Bandung'},
    'switch_to_order_menu': 0,
    'cuisines': 'Western, Asian',
    'timings': '7 AM to 10 PM',
    'average_cost_for_two': 200000,
    'price_range': 3,
    'currency': 'IDR',
    'highlights': ['Dinner',
     'Cash',
     'Credit Card',

In [53]:
def scrapeZomato(city_id, cuisine, category, establishment_type):
    zomato_temp = []
    result_founds = getJSON(city = city_id,
                            cui = cuisine, 
                            cat = category,
                            est = establishment_type,
                            result = 'results_found')
    repeat = result_founds//20 + 1
    repeat_limited = repeat if repeat < 5 else 5
    #print(city_id, cuisine, category, establishment_type, result_founds)
    for i in range(repeat_limited):
        start_ = 20 * i
        json_results = getJSON(city = city_id,
                               cui = cuisine,
                               start = start_,
                               cat = category,
                               est = establishment_type)
        for resto in json_results:
            id_res = resto['restaurant']['id']
            nama = resto['restaurant']['name']
            menu = resto['restaurant']['cuisines']
            rate = resto['restaurant']['user_rating']
            rating = rate['aggregate_rating']
            kategori = rate['rating_text']
            vote = rate['votes'] 
            locate = resto['restaurant']['location']
            city = locate['city']
            location = locate['address']
            latitude = locate['latitude']
            longitude = locate['longitude']
            buka = resto['restaurant']['timings']
            dict_temp = {'id':id_res,
                         'name': nama,
                         'menu': menu, 
                         'rating': rating, 
                         'rating category': kategori, 
                         'votes': vote,
                         'city': city,
                         'location': location, 
                         'latitude': latitude,
                         'longitude': longitude,
                         'operasional': buka}
            zomato_temp.append(dict_temp)
    return pd.DataFrame(zomato_temp)

In [55]:
is_first = True
for city_id in city_ids:
    print(city_id)
    for cuisine in cuisines:
        for category in categories:
            for establishment_type in establishments:
                if is_first:
                    df_zomato = scrapeZomato(city_id, cuisine, category, establishment_type)
                    is_first = False
                else:
                    try:
                        df_temp = scrapeZomato(city_id, cuisine, category, establishment_type)
                        df_zomato = df_zomato.append(df_temp)
                        df_zomato.to_pickle("all_zomato.pkl")
                    except:
                        pass

11052


In [58]:
all_zomato = pd.read_pickle("all_zomato.pkl").drop_duplicates()
all_zomato.shape

(7, 11)

In [40]:
df1 = pd.read_pickle
df3.reset_index().to_csv("all_zomato.csv")

In [59]:
def getJSON(city, start = 0, result= 'restaurants'):
    url = f'https://developers.zomato.com/api/v2.1/search?city_id={city}&start={start}&count=20'
    data = requests.get(url, headers=headInfo)
    return data.json()[result]

In [67]:
def scrapeZomato(city_id):
    zomato_temp = []
    result_founds = getJSON(city = city_id,
                            result = 'results_found')
    repeat = result_founds//20 + 1
    repeat_limited = repeat if repeat < 5 else 5
    #print(city_id, cuisine, category, establishment_type, result_founds)
    for i in range(repeat_limited):
        start_ = 20 * i
        json_results = getJSON(city = city_id,
                               start = start_)
        for resto in json_results:
            id_res = resto['restaurant']['id']
            nama = resto['restaurant']['name']
            menu = resto['restaurant']['cuisines']
            rate = resto['restaurant']['user_rating']
            rating = rate['aggregate_rating']
            kategori = rate['rating_text']
            vote = rate['votes'] 
            locate = resto['restaurant']['location']
            city = locate['city']
            location = locate['address']
            latitude = locate['latitude']
            longitude = locate['longitude']
            buka = resto['restaurant']['timings']
            dict_temp = {'id':id_res,
                         'name': nama,
                         'menu': menu, 
                         'rating': rating, 
                         'rating category': kategori, 
                         'votes': vote,
                         'city': city,
                         'location': location, 
                         'latitude': latitude,
                         'longitude': longitude,
                         'operasional': buka}
            zomato_temp.append(dict_temp)
    return pd.DataFrame(zomato_temp)

In [77]:
df_bandung = scrapeZomato(11052).drop_duplicates()
df_bandung_all = df_bandung.append(all_zomato).drop_duplicates()

In [79]:
df_all = pd.read_csv("all_zomato.csv").iloc[:,2:]
new_df_all = df_all.append(df_bandung_all)
# df_bandung_all

In [80]:
new_df_all.to_csv("all_zomato_new.csv")